In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.optim as optim

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import data as dt, neuronshap as ns, sim
from cfgs.fedargs import *

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
    false_negative_rate,
    false_positive_rate,
    true_negative_rate,
    true_positive_rate,
)
from libs.helpers.finance import bin_hours_per_week
from libs.helpers.metrics import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from libs.helpers.plot import group_box_plots

/Users/hkasyap/anaconda3/envs/menv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [3]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")


def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

In [4]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [5]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [6]:
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [7]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

In [8]:
train["race"].value_counts()

white                 25933
black                  2817
asian_pac_islander      895
amer_indian_eskimo      286
other                   231
Name: race, dtype: int64

In [9]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [10]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [11]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
data_dir = "../../data/adult"

In [13]:
original_features = cts_features + one_hot_features + binary_features

train_df[original_features].to_csv("../../data/adult/train.csv", index=False)
val_df[original_features].to_csv("../../data/adult/val.csv", index=False)
test_df[original_features].to_csv("../../data/adult/test.csv", index=False)

In [14]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

In [15]:
train_df.drop(columns=one_hot_features).to_csv("../../data/adult/train-one-hot.csv", index=False)
val_df.drop(columns=one_hot_features).to_csv("../../data/adult/val-one-hot.csv", index=False)
test_df.drop(columns=one_hot_features).to_csv("../../data/adult/test-one-hot.csv", index=False)

In [16]:
train = pd.read_csv("../../data/adult/train.csv")
val = pd.read_csv("../../data/adult/val.csv")
test = pd.read_csv("../../data/adult/test.csv")

train_oh = pd.read_csv("../../data/adult/train-one-hot.csv")
val_oh = pd.read_csv("../../data/adult/val-one-hot.csv")
test_oh = pd.read_csv("../../data/adult/test-one-hot.csv")

In [17]:
test_oh.head(5)

,age,sex,capital_gain,capital_loss,hours_per_week,salary,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
0,-1.015917,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,-0.029378,1,-0.147741,-0.218133,0.752765,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.788255,1,-0.147741,-0.218133,-0.079269,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0.425948,1,0.872159,-0.218133,-0.079269,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,-0.332929,1,-0.147741,-0.218133,-0.911303,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [18]:
#https://github.com/ritvikkhanna09/Census-classifier-comparison

In [19]:
'''
race_amer_indian_eskimo               15060 non-null  int64  
race_asian_pac_islander               15060 non-null  int64  
race_black                            15060 non-null  int64  
race_other                            15060 non-null  int64  
race_white
 
'''

mr_dh_oh = test_oh.loc[(test_oh["race_asian_pac_islander"] == 1) | (test_oh["race_white"] == 1)]
mr_dh_oh = mr_dh_oh.head(100)
fmr_dh_oh = test_oh.loc[(test_oh["race_amer_indian_eskimo"] == 1) | (test_oh["race_black"] == 1) | (test_oh["race_other"] == 1)]
fmr_dh_oh = fmr_dh_oh.head(100)


m_dh_oh = test_oh.loc[test_oh["sex"] == 1]
m_dh_oh = m_dh_oh.head(100)
fm_dh_oh = test_oh.loc[test_oh["sex"] == 0]
fm_dh_oh = fm_dh_oh.head(100)

In [20]:
X_train = train_oh.drop(columns="salary").values
Y_train = train_oh['salary'].values
X_test = test_oh.drop(columns="salary").values
Y_test = test_oh['salary'].values
X_m = m_dh_oh.drop(columns="salary").values
Y_m = m_dh_oh['salary'].values
X_fm = fm_dh_oh.drop(columns="salary").values
Y_fm = fm_dh_oh['salary'].values

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(Y_train).long())
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test).float(), torch.tensor(Y_test).long())
m_data = torch.utils.data.TensorDataset(torch.tensor(X_m).float(), torch.tensor(Y_m).long())
fm_data = torch.utils.data.TensorDataset(torch.tensor(X_fm).float(), torch.tensor(Y_fm).long())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
m_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fm_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)
mr_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fmr_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)

In [21]:
class BasicNet(torch.nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(50, 50)        
        self.lin3 = torch.nn.Linear(50, 50)
        
        self.lin4 = torch.nn.Linear(150, 150) 
        
        self.lin5 = torch.nn.Linear(50, 50)        
        self.lin6 = torch.nn.Linear(50, 50)
        self.lin10 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, xin):
        self.layers = 0
        
        x = F.relu(self.lin1(xin))
        self.layers += 1
        
        #x = F.relu(self.lin2(x))
        #self.layers += 1
        for y in range(8):
            x = F.relu(self.lin4(x)) 
            self.layers += 1
           
        x = self.dropout(x)
        
        x = F.relu(self.lin10(x)) 
        self.layers += 1
        return x

In [22]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
      
        #inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target.long())
        # Backprop
        loss.backward()
        optimizer.step()
        ###

In [23]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            
            #inputs, target = inputs.to(device), target.to(device)
            
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target.long()).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [24]:
model = BasicNet(63, 2)
test_accuracy = []
train_loss = []
nbr_epochs = 5
lr = 0.0025# 
weight_decay = 0

# Surrogate loss used for training
loss_fn = torch.nn.CrossEntropyLoss()
test_loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Training beginning...')
#start_time = time.time()

for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
    # save results every epoch
    test_accuracy.append(acc)
    train_loss.append(loss)
    
#end_time = time.time()
#print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :

Test set: Average loss: 0.3240, Accuracy: 12780/15060 (85%)

Epoch  2 :

Test set: Average loss: 0.3444, Accuracy: 12662/15060 (84%)

Epoch  3 :

Test set: Average loss: 0.3283, Accuracy: 12571/15060 (83%)

Epoch  4 :

Test set: Average loss: 0.3218, Accuracy: 12703/15060 (84%)

Epoch  5 :

Test set: Average loss: 0.3326, Accuracy: 12749/15060 (85%)



In [25]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 12749/15060 (85%)

3700 tensor([3645]) tensor([3645])


<h1>Demographic Parity</h1>

<h2>Distribution of scores by sex</h2>

In [50]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.071
Demographic parity ratio: 0.366


<h2>Distribution of scores by race</h2>

In [51]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.114
Demographic parity ratio: 0.105


<h1>Conditional Demographic Parity</h1>

<h2>Distribution of scores by sex and hours worked per week</h2>

In [52]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.059
Conditional demographic parity ratio: 0.627


<h2>Distribution of scores by race and hours worked per week</h2>

In [53]:
cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.175
Conditional demographic parity ratio: 0.058


<h1>Equalised Odds</h1>

<h2>Distribution of scores by sex for high and low earners</h2>

In [54]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.011
Equalised odds ratio: 0.319


<h2>Distribution of scores by race for high and low earners</h2>

In [55]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.250
Equalised odds ratio: 0.000


<h1>Shapley based Neuron Pruning for Fairness</h1>

In [32]:
m_shapley_values = ns.calculate_shapley_values_fa(model, m_loader, 100)
print(m_shapley_values)
fm_shapley_values = ns.calculate_shapley_values_fa(model, fm_loader, 100)
print(fm_shapley_values)

[  52.877377   77.55837    16.379549 ... 3388.5007   1607.3066
    0.      ]
[  49.849373    0.         14.680171 ... 3549.225     531.03326
    0.      ]


In [46]:
diff_shap_values = m_shapley_values - fm_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -150)[-150:]
diff_shap_values[max_diff_shap_values_ind]

array([  71.62713 ,   71.912186,   71.67966 ,   72.16496 ,   76.35712 ,
         73.06438 ,   73.455605,   72.96662 ,   73.79617 ,   76.343445,
         72.58319 ,   75.73064 ,   76.61609 ,   96.18156 ,   89.79048 ,
         88.41321 ,   81.17374 ,   88.33698 ,   92.27591 ,   76.83716 ,
         81.58118 ,   88.52043 ,   99.85916 ,   83.91992 ,   78.1815  ,
         88.490036,   90.81383 ,   83.61939 ,   76.74428 ,   77.55837 ,
         92.45502 ,   94.560814,   77.05978 ,   88.72458 ,   97.26541 ,
         85.29304 ,   85.1465  ,   90.27347 ,  103.81008 ,   93.79936 ,
         81.38835 ,   94.92759 ,   95.455826,   90.86333 ,   87.24123 ,
         99.99918 ,   86.26289 ,   89.44297 ,   94.014404,   92.83373 ,
         91.95735 ,  101.382454,   94.18377 ,   79.73165 ,   81.64512 ,
         91.42771 ,   88.085495,   95.64511 ,   85.33082 ,   79.674385,
         92.457306,   97.00473 ,   97.52826 ,   93.462715,   78.68754 ,
        101.50879 ,   99.74773 ,   80.26223 ,   89.34128 ,   82.

In [47]:
model_arr, model_slist = sim.get_net_arr(model)
model_arr[max_diff_shap_values_ind] = 0
updated_model = sim.get_arr_net(model, model_arr, model_slist)

In [48]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = updated_model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))
        

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 12425/15060 (83%)

3700 tensor([1343]) tensor([1343])


In [49]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.071
Demographic parity ratio: 0.366


In [37]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.121
Conditional demographic parity ratio: 0.526


In [38]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.046
Equalised odds ratio: 0.295


In [39]:
1. 

1.0